In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import glob
import pandas as pd
import numpy as np
from sglm.helpers import filehelpers as fh, dfhelpers as dfh
from pathlib import Path
from collections import defaultdict
from tqdm.notebook import tqdm, trange
import scipy.stats
import matplotlib.pyplot as plt


In [4]:
# base_location = Path(r'/Users/josh/Documents/Harvard/GLM/sabatinilab-glm/sglm/outputs')
base_location = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\outputs')

# fig_folder_lst = [('Figure_1_2', 'f1*-ft'),
#                   ('Figure_3',   'f3*-ft'),
#                   ('Figure_4_g1',   'f4*-ft'),
#                   ('Figure_4_g2',   'f4*-ft'),
#                   ('Figure_5_g1',   'f5*-ft'),
#                   ('Figure_5_g2',   'f5*-ft'),
#                   ('Figure_5_g5',   'f5*-ft'),
#                  ]
fig_folder_lst = [#('Figure_1_2', 'f1-n*-ft'),
#                   ('Figure_3',   'f3*-ft'),
#                   ('Figure_4_g1',   'f4*-ft'),
                  ('Figure_4_g2',   'f4*-ft'),
#                   ('Figure_5_g1',   'f5*-ft'),
#                   ('Figure_5_g2',   'f5*-ft'),
#                   ('Figure_5_g5',   'f5*-ft'),
                 ]

for fig_folder in fig_folder_lst:
    print('fig_folder', fig_folder)
    
    load_folder = base_location / Path(fig_folder[0])
    h5_recons_locations = str((load_folder / Path('all') / Path(fig_folder[1]) / Path(r'reconstructs\best_resids_*.h5')).resolve())
    
    out_folder = base_location / Path(r'final_outputs_rev-msesplt')

    out_loc = str((out_folder).resolve())
    h5_recons_locations = glob.glob(h5_recons_locations)

    fh.create_folder_if_not_exists(out_loc)
    
    h5_lst = defaultdict(dict)

    for ih5, h5_coef_fn in enumerate(h5_recons_locations):
        h5f = pd.HDFStore(h5_coef_fn)
        h5fk = h5f.keys()
        for ik, key in enumerate(h5fk):
            key = key.replace('/', '')
            resp = key.split('_')[0]
            model_version = '_'.join(key.split('_')[3:]).split('_run_num')[0]
            if key != 'dfrel_basis':
                
                continue

            if 'run_num' in key and 'run_num__0' not in key: #and 'run_num__1' not in key:
                continue
            if 'base_simple' not in model_version and 'base_words' not in model_version and '_to_' not in model_version and 'basis' not in key:
                continue
            
            y_col = key.split('_')[0].replace(r'/', r'')
            
            h5_df = pd.read_hdf(h5f, key)

            h5_df.columns = [dfh.shorten_col_name(_) for _ in h5_df.columns]

        h5_res = pd.read_hdf(h5f, h5fk[1])
        break
    break

fig_folder ('Figure_4_g2', 'f4*-ft')


In [5]:
holdout_name = [_ for _ in h5_res if 'holdout' in _][0]
holdout_name

'holdout_gDA_0_0_base_simple_run_num__0'

In [6]:
list(h5_df.columns)

['signal_file',
 'file_num',
 'nTrial',
 'nEndTrial',
 'wi_trial_keep',
 'nTrial_hard',
 'nEndTrial_hard',
 'diffTrialNums_hard',
 'wi_trial_keep_hard',
 'has_all_cols',
 'gDA',
 'gACH',
 'rDA',
 'diffTrialNums',
 'dupe',
 'CI',
 'CO',
 'SIr',
 'SInr',
 'SO',
 'spnnrOff',
 'sl',
 'SIAA',
 'SIAa',
 'SIaA',
 'SIaa',
 'SIAB',
 'SIAb',
 'SIaB',
 'SIab']

In [7]:
combo = h5_df.join(h5_res)[['dupe', 'file_num', 'has_all_cols', 'nTrial', 'nEndTrial', 'diffTrialNums', 'wi_trial_keep', holdout_name]].dropna()
combo = combo[combo['wi_trial_keep']&combo['has_all_cols']].reset_index()

In [8]:
combo['dupe'].sum()/(~combo['dupe']).sum()

0.015259094878307879

In [9]:
df_d = combo[combo['dupe']].copy()
df_d['align'] = df_d['nTrial_filenum'] + 1
df_dnT = df_d['align'].unique()

df_d2 = combo[combo['nTrial_filenum'].isin(df_dnT)].copy()
df_d2['align'] = df_d2['nTrial_filenum']

df_d = df_d[['nTrial_filenum', holdout_name, 'align']].drop_duplicates()
df_d2 = df_d2[['nTrial_filenum', holdout_name, 'align']].drop_duplicates()

In [10]:
df_d

,nTrial_filenum,holdout_gDA_0_0_base_simple_run_num__0,align
937,11,1.0,12
1166,14,1.0,15
1238,15,0.0,16
3182,36,0.0,37
3617,41,0.0,42
...,...,...,...
815104,32250,1.0,32251
815265,32251,0.0,32252
815481,32253,1.0,32254
815707,32255,0.0,32256


In [11]:
df_d2

,nTrial_filenum,holdout_gDA_0_0_base_simple_run_num__0,align
944,12,1.0,12
1170,15,0.0,15
1243,16,1.0,16
3190,37,1.0,37
3619,42,1.0,42
...,...,...,...
815124,32251,0.0,32251
815268,32252,0.0,32252
815496,32254,0.0,32254
815729,32256,1.0,32256


In [12]:
m = df_d.merge(df_d2, on='align')

In [13]:
m.groupby([f'{holdout_name}_x', f'{holdout_name}_y']).count()

nTrial_filenum_x  \
holdout_gDA_0_0_base_simple_run_num__0_x holdout_gDA_0_0_base_simple_run_num__0_y                     
0.0                                      0.0                                                    377   
                                         1.0                                                    364   
1.0                                      0.0                                                    343   
                                         1.0                                                    364   

                                                                                   align  \
holdout_gDA_0_0_base_simple_run_num__0_x holdout_gDA_0_0_base_simple_run_num__0_y          
0.0                                      0.0                                         377   
                                         1.0                                         364   
1.0                                      0.0                                         343   
                                         1.0                                         364   

                                                                                   nTrial_filenum_y  
holdout_gDA_0_0_base_simple_run_num__0_x holdout_gDA_0_0_base_simple_run_num__0_y                    
0.0                                      0.0                                                    377  
                                         1.0                                                    364  
1.0                                      0.0                                                    343  
                                         1.0                                                    364

In [14]:
# list(h5_df.columns)

In [15]:
# combo

In [16]:
# combo

In [17]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)